Cross validation of parameters for the genetic algorithm

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Updated simulated annealing experiment runner.

Changes:
- Added plot_scheduler_comparison_detailed(...) to produce scheduler_comparison_detailed.png
  that compares ALL experiment names (Geometric_Fast, Geometric_Slow, ...) for Random vs Local.
- Added save_top_n_runs(...) to save the top N runs (defaults to 10) in the layout/dataset folder.
  Each top run gets an individual detailed txt file and there's a top_10_summary.txt with a table.
- Results now include 'initial_layout_name' in each result so the top-run dumps include the initial
  layout info the run started from.

Usage: same as original. Replace your previous script with this one.
"""

import numpy as np
import random
import time
from collections import defaultdict
import os
import matplotlib.pyplot as plt
from pathlib import Path

# Keyboard Layouts
qwerty = ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p',
        'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';',
        'z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', "'"]
dvorak = ["'", ',', '.', 'p', 'y', 'f', 'g', 'c', 'r', 'l',
        'a', 'o', 'e', 'u', 'i', 'd', 'h', 't', 'n', 's',
        ';', 'q', 'j', 'k', 'x', 'b', 'm', 'w', 'v', 'z']
qwertz = ['q', 'w', 'e', 'r', 't', 'z', 'u', 'i', 'o', 'p',
        'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';',
        'y', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', "'"]
colemak = ['q', 'w', 'f', 'p', 'g', 'j', 'l', 'u', 'y', ';',
        'a', 'r', 's', 't', 'd', 'h', 'n', 'e', 'i', 'o',
        'z', 'x', 'c', 'v', 'b', 'k', 'm', ',', '.', "'"]

exp1_best_moby_dick = ['x', 'v', 'w', 'c', 'h', 'i', 't', 'k', 'z', "'", 'f', 'm', 'r', 's', 'n', 'e', 'a', 'o', ',', '.', 'q', 'b', 'p', 'l', 'd', 'g', 'u', 'y', ';', 'j']
exp2_best_moby_dick = ['x', 'v', 'w', 'c', 'h', 'i', 't', 'k', 'z', "'", 'f', 'm', 'r', 's', 'n', 'e', 'a', 'o', ',', '.', 'q', 'b', 'p', 'l', 'd', 'g', 'u', 'y', ';', 'j']
exp3_best_moby_dick = ['x', 'v', 'w', 'c', 'h', 'i', 't', 'k', 'z', "'", 'f', 'm', 'r', 's', 'n', 'e', 'a', 'o', ',', '.', 'q', 'b', 'p', 'l', 'd', 'g', 'u', 'y', ';', 'j']
exp4_best_moby_dick = ["'", 'z', 'w', 't', 'i', 'h', 'c', 'f', 'k', 'x', '.', ',', 'o', 'a', 'e', 'n', 's', 'r', 'm', 'v', 'j', ';', 'y', 'u', 'g', 'd', 'l', 'p', 'b', 'q']
ga_best_moby_dick = ["'", 'z', 'w', 't', 'i', 'h', 'c', 'f', 'k', 'x', '.', ',', 'o', 'a', 'e', 'n', 's', 'r', 'm', 'v', 'j', ';', 'y', 'u', 'g', 'd', 'l', 'p', 'b', 'q']

exp1_best_wizard_oz = ['z', 'k', 'w', 's', 'h', 'i', 't', 'c', ',', ';', 'v', 'd', 'n', 'l', 'r', 'e', 'a', 'o', 'y', '.', 'q', 'x', 'b', 'f', 'm', 'g', 'u', 'p', 'j', "'"]
exp2_best_wizard_oz = ['x', 'v', 'c', 'w', 'h', 'i', 't', 'k', 'z', "'", 'f', 'm', 'r', 's', 'n', 'e', 'a', 'o', 'g', '.', 'q', 'b', 'p', 'l', 'd', ',', 'y', 'u', 'j', ';']
exp3_best_wizard_oz = ['q', 'p', 'u', 't', 'i', 'h', 's', 'c', 'b', 'x', '.', 'y', 'o', 'a', 'e', 'n', 'r', 'l', 'm', 'v', ';', 'j', ',', 'g', 'k', 'w', 'd', 'f', 'z', "'"]
exp4_best_wizard_oz = ['j', ';', 'p', 'y', ',', 'd', 'l', 'm', 'b', "'", '.', 'u', 'o', 'a', 'e', 'n', 's', 'r', 'f', 'v', 'q', 'z', 'g', 't', 'i', 'h', 'w', 'c', 'k', 'x']
ga_best_wizard_oz = ['j', ';', 'p', 'y', ',', 'd', 'l', 'm', 'b', "'", '.', 'u', 'o', 'a', 'e', 'n', 's', 'r', 'f', 'v', 'q', 'z', 'g', 't', 'i', 'h', 'w', 'c', 'k', 'x']

def get_finger_assigned(position):
    row = position // 10
    col = position % 10
    finger_map = {
        0: ('L', 0, 1), 1: ('L', 1, 2), 2: ('L', 2, 3), 3: ('L', 3, 4), 4: ('L', 3, 4),
        5: ('R', 3, 4), 6: ('R', 3, 4), 7: ('R', 2, 3), 8: ('R', 1, 2), 9: ('R', 0, 1)
    }
    hand, finger, strength = finger_map[col]
    return hand, finger, strength, row, col

def euclidean_distance(pos1, pos2):
    _, _, _, row1, col1 = get_finger_assigned(pos1)
    _, _, _, row2, col2 = get_finger_assigned(pos2)
    return np.sqrt((row1 - row2) ** 2 + (col1 - col2) ** 2)

def finger_penalty(pos1, pos2):
    hand1, finger1, strength1, row1, col1 = get_finger_assigned(pos1)
    hand2, finger2, strength2, row2, col2 = get_finger_assigned(pos2)

    penalty = 0
    if hand1 == hand2 and finger1 == finger2 and pos1 != pos2:
        penalty += 1.0
        if strength1 <= 2:
            penalty += 2.0
    elif hand1 == hand2:
        penalty += 1.0
    penalty += -1.0

    row_diff = abs(row1 - row2)
    if row_diff == 1:
        penalty += 0.2
        if strength1 <= 2 or strength2 <= 2:
            penalty += 0.15
    elif row_diff == 2:
        penalty += 0.8
        if strength1 <= 2 or strength2 <= 2:
            penalty += 0.5

    if finger1 == 0 or finger2 == 0:
        penalty += 0.15
    if finger1 == 1 or finger2 == 1:
        penalty += 0.1

    if col1 == col2 and row_diff > 0:
        penalty += 0.3
        if col1 in [0, 9]:
            penalty += 0.2
        elif col1 in [1, 8]:
            penalty += 0.1

    return penalty

def precompute_cost_matrices():
    distances = np.zeros((30, 30))
    penalties = np.zeros((30, 30))

    for i in range(30):
        for j in range(30):
            distances[i, j] = euclidean_distance(i, j)
            penalties[i, j] = finger_penalty(i, j)

    return distances, penalties

DISTANCE_MATRIX, PENALTY_MATRIX = precompute_cost_matrices()

def load_text_from_file(filename, sample_size=None):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            text = file.read().lower()

        if sample_size and sample_size < len(text):
            text = text[:sample_size]

        return text
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return None
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        return None

def precompute_bigrams(text):
    bigrams = defaultdict(int)

    for i in range(len(text) - 1):
        bigram = (text[i], text[i + 1])
        bigrams[bigram] += 1

    return dict(bigrams)

def fitness_function(keyboard, bigram_freq):
    pos_map = {char: idx for idx, char in enumerate(keyboard)}
    total_cost = 0

    for (char1, char2), freq in bigram_freq.items():
        if char1 not in pos_map or char2 not in pos_map:
            continue

        pos1 = pos_map[char1]
        pos2 = pos_map[char2]

        if pos1 == pos2:
            continue

        base_dist = DISTANCE_MATRIX[pos1, pos2]
        finger_cost = PENALTY_MATRIX[pos1, pos2]
        total_multiplier = max(1.0 + finger_cost, 0.1)

        total_cost += (base_dist * total_multiplier) * freq

    return total_cost

def get_random_neighbor(layout):
    neighbor = layout.copy()
    pos1, pos2 = np.random.choice(30, 2, replace=False)
    neighbor[pos1], neighbor[pos2] = neighbor[pos2], neighbor[pos1]
    return neighbor

def get_local_neighbor(layout):
    neighbor = layout.copy()
    pos1 = np.random.randint(0, 30)

    row = pos1 // 10
    col = pos1 % 10

    adjacent_positions = []
    if col > 0:
        adjacent_positions.append(pos1 - 1)
    if col < 9:
        adjacent_positions.append(pos1 + 1)
    if row > 0:
        adjacent_positions.append(pos1 - 10)
    if row < 2:
        adjacent_positions.append(pos1 + 10)

    if adjacent_positions:
        pos2 = random.choice(adjacent_positions)
        neighbor[pos1], neighbor[pos2] = neighbor[pos2], neighbor[pos1]

    return neighbor

def temperature_schedule(T_initial, iteration, schedule_type='geometric', k=0.95):
    if schedule_type == 'linear':
        return max(T_initial - iteration * k, 0.01)
    elif schedule_type == 'geometric':
        return T_initial * (k ** iteration)
    elif schedule_type == 'logarithmic':
        return T_initial / (1 + k * T_initial * iteration)
    else:
        raise ValueError(f"Unknown schedule type: {schedule_type}")

def simulated_annealing(text_file, initial_layout_spec='random', T_initial=1000,
                       max_iterations=100000, schedule_type='geometric', k=0.95,
                       neighbor_method='random', seed=None, verbose=False):

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    text = load_text_from_file(text_file)
    if text is None:
        return None, None

    bigram_freq = precompute_bigrams(text)

    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
            ',', '.', ';', "'"]

    if initial_layout_spec == 'random':
        S_actual = random.sample(letters, len(letters))
    elif isinstance(initial_layout_spec, list):
        S_actual = initial_layout_spec.copy()
    else:
        S_actual = random.sample(letters, len(letters))

    S_mejor = S_actual.copy()
    f_actual = fitness_function(S_actual, bigram_freq)
    f_mejor = f_actual
    f_inicial = f_actual

    neighbor_func = get_local_neighbor if neighbor_method == 'local' else get_random_neighbor

    history = {
        'iteration': [],
        'temperature': [],
        'current_fitness': [],
        'best_fitness': [],
        'accepted_moves': 0,
        'rejected_moves': 0
    }

    for i in range(max_iterations):
        T = temperature_schedule(T_initial, i, schedule_type, k)

        S_nuevo = neighbor_func(S_actual)
        f_nuevo = fitness_function(S_nuevo, bigram_freq)

        delta_f = f_actual - f_nuevo

        if delta_f > 0:
            S_actual = S_nuevo
            f_actual = f_nuevo
            history['accepted_moves'] += 1

            if f_nuevo < f_mejor:
                S_mejor = S_nuevo.copy()
                f_mejor = f_nuevo
        else:
            if T > 0:
                acceptance_prob = np.exp(delta_f / T)
                if random.random() < acceptance_prob:
                    S_actual = S_nuevo
                    f_actual = f_nuevo
                    history['accepted_moves'] += 1
                else:
                    history['rejected_moves'] += 1
            else:
                history['rejected_moves'] += 1

        if i % 100 == 0 or i == max_iterations - 1:
            history['iteration'].append(i)
            history['temperature'].append(T)
            history['current_fitness'].append(f_actual)
            history['best_fitness'].append(f_mejor)

    history['initial_fitness'] = f_inicial
    history['final_fitness'] = f_mejor
    history['improvement'] = f_inicial - f_mejor
    history['improvement_pct'] = ((f_inicial - f_mejor) / f_inicial) * 100 if f_inicial > 0 else 0

    return S_mejor, history

def format_keyboard_layout(layout, title="Keyboard Layout"):
    result = f"\n{title}\n"
    result += "Row 1: " + " ".join(layout[0:10]) + "\n"
    result += "Row 2: " + " ".join(layout[10:20]) + "\n"
    result += "Row 3: " + " ".join(layout[20:30]) + "\n"
    return result

def plot_fitness_evolution(results, output_path):
    plt.figure(figsize=(12, 6))

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']

    for idx, result in enumerate(results):
        history = result['history']
        plt.plot(history['iteration'], history['best_fitness'],
                color=colors[idx % len(colors)], linewidth=2,
                label=result['name'], alpha=0.7)

    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Best Fitness', fontsize=12)
    plt.title('Fitness Evolution', fontsize=14, fontweight='bold')
    plt.legend(fontsize=9, loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

def plot_scheduler_comparison(results_random, results_local, output_path):
    schedulers = {}
    for result in results_random:
        scheduler = result['config']['schedule_type']
        if scheduler not in schedulers:
            schedulers[scheduler] = {'random': [], 'local': []}
        schedulers[scheduler]['random'].append(result['final_fitness'])

    for result in results_local:
        scheduler = result['config']['schedule_type']
        if scheduler not in schedulers:
            schedulers[scheduler] = {'random': [], 'local': []}
        schedulers[scheduler]['local'].append(result['final_fitness'])

    scheduler_names = list(schedulers.keys())
    random_avgs = [np.mean(schedulers[s]['random']) if schedulers[s]['random'] else 0 for s in scheduler_names]
    local_avgs = [np.mean(schedulers[s]['local']) if schedulers[s]['local'] else 0 for s in scheduler_names]

    x = np.arange(len(scheduler_names))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))

    bars1 = ax.bar(x - width/2, random_avgs, width, label='Random Swap', color='#1f77b4', alpha=0.8)
    bars2 = ax.bar(x + width/2, local_avgs, width, label='Local Swap', color='#ff7f0e', alpha=0.8)

    ax.set_xlabel('Temperature Scheduler', fontsize=12)
    ax.set_ylabel('Average Final Fitness', fontsize=12)
    ax.set_title('Scheduler Comparison: Random vs Local Neighborhood', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(scheduler_names, rotation=45, ha='right')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, axis='y')

    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:,.0f}',
                   ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

def plot_scheduler_comparison_detailed(results_random, results_local, output_path):
    """
    New detailed scheduler comparison.
    X axis: experiment names (e.g., Geometric_Fast, Geometric_Slow, ...)
    For each experiment name, plot average final fitness for Random swap and Local swap.
    """
    experiments = {}
    # Aggregate by experiment name
    for result in results_random:
        name = result['name']
        if name not in experiments:
            experiments[name] = {'random': [], 'local': []}
        experiments[name]['random'].append(result['final_fitness'])

    for result in results_local:
        name = result['name']
        if name not in experiments:
            experiments[name] = {'random': [], 'local': []}
        experiments[name]['local'].append(result['final_fitness'])

    exp_names = sorted(experiments.keys())

    random_avgs = [np.mean(experiments[n]['random']) if experiments[n]['random'] else 0 for n in exp_names]
    local_avgs = [np.mean(experiments[n]['local']) if experiments[n]['local'] else 0 for n in exp_names]

    x = np.arange(len(exp_names))
    width = 0.35

    fig, ax = plt.subplots(figsize=(14, 7))
    bars1 = ax.bar(x - width/2, random_avgs, width, label='Random Swap', color='#1f77b4', alpha=0.9)
    bars2 = ax.bar(x + width/2, local_avgs, width, label='Local Swap', color='#ff7f0e', alpha=0.9)

    ax.set_xlabel('Experiment (Scheduler configuration)', fontsize=12)
    ax.set_ylabel('Average Final Fitness', fontsize=12)
    ax.set_title('Detailed Scheduler Comparison by Experiment (Random vs Local)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(exp_names, rotation=45, ha='right', fontsize=9)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.25, axis='y')

    for bar in bars1 + bars2:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2., height,
                f'{height:,.0f}', ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()

def save_comprehensive_results(results_random, results_local, initial_layout_name, dataset_name, output_dir):
    filepath = output_dir / "results.txt"

    with open(filepath, 'w', encoding='utf-8') as f:
        f.write("=" * 100 + "\n")
        f.write(f"SIMULATED ANNEALING RESULTS\n")
        f.write(f"Initial Layout: {initial_layout_name}\n")
        f.write(f"Dataset: {dataset_name}\n")
        f.write("=" * 100 + "\n\n")

        f.write("RANDOM SWAP RESULTS\n")
        f.write("=" * 100 + "\n")
        for result in sorted(results_random, key=lambda x: x['final_fitness']):
            f.write(f"\nExperiment: {result['name']}\n")
            f.write(f"Schedule: {result['config']['schedule_type']}, T_initial: {result['config']['T_initial']}, k: {result['config']['k']}\n")
            f.write(f"Initial Fitness: {result['history']['initial_fitness']:,.2f}\n")
            f.write(f"Final Fitness: {result['final_fitness']:,.2f}\n")
            f.write(f"Improvement: {result['history']['improvement_pct']:.2f}%\n")
            f.write(f"Time: {result['time']:.2f}s\n")
            f.write(format_keyboard_layout(result['layout'], "Final Layout"))
            f.write("\n")

        f.write("\n" + "=" * 100 + "\n")
        f.write("LOCAL SWAP RESULTS\n")
        f.write("=" * 100 + "\n")
        for result in sorted(results_local, key=lambda x: x['final_fitness']):
            f.write(f"\nExperiment: {result['name']}\n")
            f.write(f"Schedule: {result['config']['schedule_type']}, T_initial: {result['config']['T_initial']}, k: {result['config']['k']}\n")
            f.write(f"Initial Fitness: {result['history']['initial_fitness']:,.2f}\n")
            f.write(f"Final Fitness: {result['final_fitness']:,.2f}\n")
            f.write(f"Improvement: {result['history']['improvement_pct']:.2f}%\n")
            f.write(f"Time: {result['time']:.2f}s\n")
            f.write(format_keyboard_layout(result['layout'], "Final Layout"))
            f.write("\n")

        best_random = min(results_random, key=lambda x: x['final_fitness']) if results_random else None
        best_local = min(results_local, key=lambda x: x['final_fitness']) if results_local else None

        f.write("\n" + "=" * 100 + "\n")
        f.write("BEST RESULTS SUMMARY\n")
        f.write("=" * 100 + "\n")

        if best_random:
            f.write(f"\nBest Random Swap:\n")
            f.write(f"Experiment: {best_random['name']}\n")
            f.write(f"Final Fitness: {best_random['final_fitness']:,.2f}\n")
            f.write(f"Improvement: {best_random['history']['improvement_pct']:.2f}%\n")
            f.write(format_keyboard_layout(best_random['layout'], "Best Random Layout"))

        if best_local:
            f.write(f"\nBest Local Swap:\n")
            f.write(f"Experiment: {best_local['name']}\n")
            f.write(f"Final Fitness: {best_local['final_fitness']:,.2f}\n")
            f.write(f"Improvement: {best_local['history']['improvement_pct']:.2f}%\n")
            f.write(format_keyboard_layout(best_local['layout'], "Best Local Layout"))

        if best_random and best_local:
            overall_best = best_random if best_random['final_fitness'] < best_local['final_fitness'] else best_local
            method = 'Random Swap' if overall_best == best_random else 'Local Swap'
            f.write(f"\nOverall Best ({method}):\n")
            f.write(f"Experiment: {overall_best['name']}\n")
            f.write(f"Final Fitness: {overall_best['final_fitness']:,.2f}\n")
            f.write(f"Improvement: {overall_best['history']['improvement_pct']:.2f}%\n")
            f.write(format_keyboard_layout(overall_best['layout'], "Overall Best Layout"))

    print(f"Results saved to: {filepath}")

def save_top_n_runs(results_random, results_local, output_dir, n=10):
    """
    Save top N runs (across both random and local) to the output_dir.
    For each top run, create a detailed text file with:
    - initial layout name
    - initial layout (if known)
    - config: schedule_type, T_initial, k
    - neighbor method (random/local)
    - initial and final fitness, improvement, time
    - final layout
    Also create a top_10_summary.txt listing the top N with brief info.
    """
    merged = []
    for r in results_random:
        rr = r.copy()
        rr['neighbor_method'] = 'random'
        merged.append(rr)
    for r in results_local:
        rr = r.copy()
        rr['neighbor_method'] = 'local'
        merged.append(rr)

    if not merged:
        return

    # sort ascending by final_fitness (lower is better)
    merged_sorted = sorted(merged, key=lambda x: x['final_fitness'])
    top_n = merged_sorted[:n]

    summary_path = output_dir / "top_10_summary.txt"
    with open(summary_path, 'w', encoding='utf-8') as s:
        s.write("TOP RUNS SUMMARY\n")
        s.write("=" * 80 + "\n")
        s.write(f"Saved top {len(top_n)} runs\n\n")
        for idx, run in enumerate(top_n, 1):
            name_safe = f"{idx}_{run['name']}_{run['neighbor_method']}"
            # brief line in summary
            s.write(f"{idx}. Experiment: {run['name']}, Method: {run['neighbor_method']}, Initial Layout: {run.get('initial_layout_name', 'N/A')}, Final Fitness: {run['final_fitness']:,.2f}, Time: {run['time']:.2f}s\n")

            # write detailed file per run
            file_name = output_dir / f"top_{name_safe}.txt"
            with open(file_name, 'w', encoding='utf-8') as f:
                f.write("=" * 80 + "\n")
                f.write(f"TOP RUN #{idx}\n")
                f.write("=" * 80 + "\n\n")
                f.write(f"Experiment name: {run['name']}\n")
                f.write(f"Neighbor method: {run.get('neighbor_method', 'N/A')}\n")
                f.write(f"Initial layout name: {run.get('initial_layout_name', 'N/A')}\n")
                # If initial layout was a list, show it
                init_spec = run.get('config', {}).get('initial_layout_spec', None)
                # we didn't store the explicit initial layout list in config; show what we can:
                if run.get('initial_layout_name') and run['initial_layout_name'] != 'Random':
                    f.write(f"Initial layout (as name): {run.get('initial_layout_name')}\n")
                else:
                    f.write(f"Initial layout spec: {run.get('initial_layout_name')}\n")
                f.write(f"\nScheduler config:\n")
                cfg = run.get('config', {})
                f.write(f"- schedule_type: {cfg.get('schedule_type')}\n")
                f.write(f"- T_initial: {cfg.get('T_initial')}\n")
                f.write(f"- k: {cfg.get('k')}\n")
                f.write("\nPerformance:\n")
                hist = run.get('history', {})
                f.write(f"- Initial fitness: {hist.get('initial_fitness', float('nan')):,.2f}\n")
                f.write(f"- Final fitness: {run.get('final_fitness', float('nan')):,.2f}\n")
                f.write(f"- Improvement: {hist.get('improvement_pct', float('nan')):.2f}%\n")
                f.write(f"- Time (s): {run.get('time', 0.0):.2f}\n")
                f.write("\nFinal layout:\n")
                f.write(format_keyboard_layout(run.get('layout', []), title="Final Layout"))
                f.write("\n")
    print(f"Top {len(top_n)} runs saved to: {summary_path}")

def main():
    datasets = {
        'moby_dick': {
            'file': 'data/moby_dick_cln.txt',
            'ga_layouts': {
                'GA_Exp1': exp1_best_moby_dick,
                'GA_Exp2': exp2_best_moby_dick,
                'GA_Exp3': exp3_best_moby_dick,
                'GA_Exp4': exp4_best_moby_dick,
                'GA_Best': ga_best_moby_dick
            }
        },
        'wizard_oz': {
            'file': 'data/wonderful_wizard_oz_cln.txt',
            'ga_layouts': {
                'GA_Exp1': exp1_best_wizard_oz,
                'GA_Exp2': exp2_best_wizard_oz,
                'GA_Exp3': exp3_best_wizard_oz,
                'GA_Exp4': exp4_best_wizard_oz,
                'GA_Best': ga_best_wizard_oz
            }
        }
    }

    initial_layouts = {
        'Random': 'random',
        'QWERTY': qwerty,
        'DVORAK': dvorak,
        'QWERTZ': qwertz,
        'COLEMAK': colemak
    }

    experiments = [
        {'name': 'Geometric_Fast', 'schedule_type': 'geometric', 'T_initial': 5000, 'k': 0.9995, 'max_iterations': 50000},
        {'name': 'Geometric_Balanced', 'schedule_type': 'geometric', 'T_initial': 10000, 'k': 0.9998, 'max_iterations': 50000},
        {'name': 'Geometric_Slow', 'schedule_type': 'geometric', 'T_initial': 15000, 'k': 0.99985, 'max_iterations': 50000},
        {'name': 'Logarithmic_Fast', 'schedule_type': 'logarithmic', 'T_initial': 5000, 'k': 0.00005, 'max_iterations': 50000},
        {'name': 'Logarithmic_Slow', 'schedule_type': 'logarithmic', 'T_initial': 10000, 'k': 0.00002, 'max_iterations': 50000},
        {'name': 'Linear_Aggressive', 'schedule_type': 'linear', 'T_initial': 10000, 'k': 0.2, 'max_iterations': 50000},
        {'name': 'Geometric_VeryHigh', 'schedule_type': 'geometric', 'T_initial': 20000, 'k': 0.9997, 'max_iterations': 50000},
        {'name': 'Geometric_UltraSlow', 'schedule_type': 'geometric', 'T_initial': 10000, 'k': 0.99995, 'max_iterations': 50000}
    ]

    output_base_dir = Path('results/sa')

    for dataset_name, dataset_info in datasets.items():
        print(f"\nProcessing dataset: {dataset_name}")

        text_file = dataset_info['file']
        if not Path(text_file).exists():
            print(f"Dataset file not found: {text_file}")
            continue

        ga_layouts = dataset_info['ga_layouts']
        all_layouts = {**initial_layouts, **ga_layouts}

        for layout_name, layout_spec in all_layouts.items():
            print(f"\nProcessing initial layout: {layout_name}")

            layout_dir = output_base_dir / dataset_name / layout_name
            layout_dir.mkdir(parents=True, exist_ok=True)

            results_random = []
            results_local = []

            for neighbor_method in ['random', 'local']:
                print(f"\nNeighbor method: {neighbor_method}")

                for exp_idx, exp_config in enumerate(experiments, 1):
                    print(f"Running experiment {exp_idx}/{len(experiments)}: {exp_config['name']}")

                    seed = hash(f"{dataset_name}_{layout_name}_{exp_config['name']}_{neighbor_method}") % (2**32)

                    time_start = time.time()
                    best_layout, history = simulated_annealing(
                        text_file=text_file,
                        initial_layout_spec=layout_spec,
                        T_initial=exp_config['T_initial'],
                        max_iterations=exp_config['max_iterations'],
                        schedule_type=exp_config['schedule_type'],
                        k=exp_config['k'],
                        neighbor_method=neighbor_method,
                        seed=seed,
                        verbose=False
                    )
                    time_end = time.time()

                    if best_layout and history:
                        result = {
                            'name': exp_config['name'],
                            # store a copy of config, include the max_iterations for record
                            'config': {**exp_config},
                            'layout': best_layout,
                            'history': history,
                            'time': time_end - time_start,
                            'final_fitness': history['final_fitness'],
                            # store which initial layout name this run started from
                            'initial_layout_name': layout_name
                        }

                        if neighbor_method == 'random':
                            results_random.append(result)
                        else:
                            results_local.append(result)

                        print(f"Completed: Final fitness = {history['final_fitness']:,.1f}")

            if results_random or results_local:
                save_comprehensive_results(results_random, results_local, layout_name, dataset_name, layout_dir)

                if results_random:
                    plot_fitness_evolution(results_random, layout_dir / "fitness_evolution_random.png")
                if results_local:
                    plot_fitness_evolution(results_local, layout_dir / "fitness_evolution_local.png")

                if results_random and results_local:
                    plot_scheduler_comparison(results_random, results_local, layout_dir / "scheduler_comparison.png")
                    # new detailed comparison (by experiment name)
                    plot_scheduler_comparison_detailed(results_random, results_local, layout_dir / "scheduler_comparison_detailed.png")

                # Save top 10 runs (across both neighbor methods)
                save_top_n_runs(results_random, results_local, layout_dir, n=10)

    print("\nAll experiments completed!")
    print(f"Results saved to: {output_base_dir.absolute()}")

if __name__ == "__main__":
    main()


Processing dataset: moby_dick

Processing initial layout: Random

Neighbor method: random
Running experiment 1/8: Geometric_Fast
Completed: Final fitness = 840,199.4
Running experiment 2/8: Geometric_Balanced
Completed: Final fitness = 778,895.1
Running experiment 3/8: Geometric_Slow
Completed: Final fitness = 774,634.3
Running experiment 4/8: Logarithmic_Fast
Completed: Final fitness = 793,208.3
Running experiment 5/8: Logarithmic_Slow
Completed: Final fitness = 781,127.6
Running experiment 6/8: Linear_Aggressive
Completed: Final fitness = 762,870.5
Running experiment 7/8: Geometric_VeryHigh
Completed: Final fitness = 788,541.5
Running experiment 8/8: Geometric_UltraSlow
Completed: Final fitness = 769,186.5

Neighbor method: local
Running experiment 1/8: Geometric_Fast
Completed: Final fitness = 1,177,847.2
Running experiment 2/8: Geometric_Balanced
Completed: Final fitness = 939,344.4
Running experiment 3/8: Geometric_Slow
Completed: Final fitness = 866,853.0
Running experiment 4/8: